# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
#| hide
from jupyter_client.manager import KernelManager
from IPython.display import display, Image
import base64
from pathlib import Path
from IPython.core.magic import register_line_magic, register_line_cell_magic

## Remote Kernel Backend

In [ ]:
#| export
#| notest
def set_ssh_config(
    port:int,                           # the port on the proxy server (bore.pub)
    user:str="",                        # username on remote machine
    alias:str="remote_server_sshpyk",   # ssh config alias; this default is expected
    proxyname:str="bore.pub",           # only ever tested with bore.pub 
    config_path:str="~/.ssh/config",    # where your ssh config file is (shouldn't be anywhere else)
    ):
    "setup ssh config file for use with `sshpyk`"
    config_path = Path(config_path).expanduser()
    if not config_path.exists(): config_path.touch()
    text = config_path.read_text()
    if f"Host {alias}" not in text: 
        assert user != "", "Must specify username when creating ~/.ssh/config info"
        block = f"""
Host {alias}
    HostName {proxyname}
    Port {port}
    User {user}
    BatchMode yes
    ControlMaster auto
    ControlPath ~/.ssh/sshpyk_%r@%h_%p
    ControlPersist 10m
    StrictHostKeyChecking no
    UserKnownHostsFile /dev/null
"""
        config_path.write_text(text + block)
    else:
        lines = text.splitlines()
        in_target_block = False
        for i, line in enumerate(lines):
            if line.startswith("Host "):
                in_target_block = (line == f"Host {alias}")
            elif in_target_block and line.strip().startswith("Port "):
                lines[i] = f"    Port {port}"
            elif proxyname and in_target_block and line.strip().startswith("HostName "):
                lines[i] = f"    HostName {proxyname}"
            elif user and in_target_block and line.strip().startswith("User "):
                lines[i] = f"    User {user}"
        config_path.write_text("\n".join(lines) + "\n")

In [ ]:
#| export
km, kc = None, None
# "ipf" = "ipyfernel" ;-) 
def ipf_startup(kernel_name="chonk_python"):  
    "Start up the remote kernel"
    global km, kc 
    if km is None and kc is None: #only do this at startup
        km = KernelManager(kernel_name=kernel_name)
        km.start_kernel()
        kc = km.client()
        kc.start_channels()
        kc.wait_for_ready(timeout=30)
        print("Success: remote kernel started")
    else: 
        print("ipf_startup: already running")

In [ ]:
def output_hook(msg):
    "Share output from the remote kernel."
    mt = msg["msg_type"]
    content = msg.get("content", {})
    if mt == "stream":
        print(content["text"], end="", flush=True)
    elif mt == "error":
        print('\n'.join(content.get("traceback", [])))
    elif mt in ("display_data", "update_display_data"):
        data = content.get("data", {})
        if "image/png" in data:
            display(Image(base64.b64decode(data["image/png"])))
        elif "text/plain" in data:
            print(data["text/plain"])

In [ ]:
#| export
@register_line_cell_magic
def local(line, cell=None):
    "local execution: works as %local and as %%local"
    get_ipython().run_cell(cell if cell else line) 

In [ ]:
#| hide

#import nbdev; nbdev.nbdev_export()
!nbdev_export --procs scrub_magics